3.
В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [18]:
from scipy import stats
import numpy as np

In [2]:
train_data = [0] * 24 + [1] * 10
test_data = [0] * 12 + [1] * 4

In [34]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [19]:
z_stat = proportions_diff_z_stat_ind(train_data, test_data)
z_stat

0.32410186177608225

In [69]:
print("p-value: %f" % round(proportions_diff_z_test(z_stat, 'greater'), 4))

p-value: 0.372900


4.
Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:

![title](CHF1000_2_back_horizontal.jpg)

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам $X_1$,$X_2$,$X_3$;

логистическая регрессия по признакам $X_4$,$X_5$,$X_6$.

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [6]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split

In [60]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in zip(sample1, sample2)])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in zip(sample1, sample2)])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in zip(sample1, sample2)])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in zip(sample1, sample2)])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [51]:
data = pd.read_csv('banknotes.txt', sep = '\t')
y = data.real
X = data.drop('real', axis = 1)

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1, shuffle = True)

In [93]:
X_train_1, X_test_1 = X_train[['X1', 'X2', 'X3']], X_test[['X1', 'X2', 'X3']]
X_train_2, X_test_2 = X_train[['X4', 'X5', 'X6']], X_test[['X4', 'X5', 'X6']]

In [94]:
model_1 = LogisticRegression(multi_class='ovr', solver='liblinear', n_jobs = 1)
model_2 = LogisticRegression(multi_class='ovr', solver='liblinear', n_jobs = 1)
model_1.fit(X_train_1, y_train)
model_2.fit(X_train_2, y_train)

LogisticRegression(multi_class='ovr', n_jobs=1, solver='liblinear')

In [95]:
print(model_1.score(X_test_1, y_test))
print(model_1.score(X_test_2, y_test))

0.8
0.46


In [96]:
pred_1 = model_1.predict(X_test_1)
pred_2 = model_1.predict(X_test_2)

In [97]:
err_1 = abs(pred_1 - y_test)
err_2 = abs(pred_2 - y_test)

In [99]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(err_1, err_2)))

p-value: 0.000042


Правильный ответ: 0.0032969384555543435  =  3

Я не смог его добиться

5.
В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [66]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(err_1, err_2))

95% confidence interval for a difference between proportions: [-0.502665, -0.177335]


Правильный ответ: 0.0599
    
Я не смог его добиться

6.
Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [84]:
z_scores = (525 - 541.4) / (100/100**0.5)
z_scores

-1.6399999999999977

In [85]:
p_val = 1 - stats.norm.cdf(abs(z_scores))
print('p_value = ', round(p_val, 4))

p_value =  0.0505


7.
Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [81]:
z_scores = (525 - 541.5) / (100/100**0.5)
z_scores

-1.65

In [82]:
p_val = 1 - stats.norm.cdf(abs(z_scores))
print('p_value = ', round(p_val, 4))

p_value =  0.0495
